In [2]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3501  100  3501    0     0   9411      0 --:--:-- --:--:-- --:--:--  9411


add /root/miniconda/lib/python3.7/site-packages to PYTHONPATH
python version: 3.7.10
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added conda-forge to channels
added omnia to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [3]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

     |████████████████████████████████| 563kB 4.2MB/s 


'2.6.0.dev'

In [4]:
import rdkit
rdkit.__version__

'2020.09.1'

In [5]:
import torch
torch.__version__

'1.8.1+cu101'

In [25]:
print(torch.cuda.device_count())
torch.cuda.get_device_name(0)

1


'Tesla P100-PCIE-16GB'

In [11]:
!pip install molsets
import moses
moses.__version__

     |████████████████████████████████| 51.6MB 100kB/s 
     |████████████████████████████████| 5.6MB 18.6MB/s 
     |████████████████████████████████| 5.2MB 49.9MB/s 


'0.3.1'

In [12]:
# LOAD THE DATASET WITH MINIMAL PRE-PROCESSING
!mkdir zinc_data

tasks, datasets, _ = deepchem.molnet.load_zinc15(
    featurizer='raw',
    splitter=None,
    transformers=[],
    data_dir='zinc_data', 
    save_dir='zinc_data',
    dataset_size='250K')
print(tasks)

['mwt', 'logp', 'reactive']


In [28]:
# EXTRACT THE SMILES STRINGS FROM THE DATASET
data = datasets[0]
train_smiles = []
for X, _, _, _ in data.itersamples():
    train_smiles.append(rdkit.Chem.MolToSmiles(X))
print(len(train_smiles))
for smile in train_smiles[0:5]:
  print(smile)

250000
CCN(CCSC)C(=O)N[C@@](C)(CC)C(F)(F)F
CC1(C)CN(C(=O)Nc2cc3ccccc3nn2)C[C@@]2(CCOC2)O1
CC[C@H](NC(C)=O)C(=O)NCC1(NC(=O)Cc2nonc2C)CC1
O=C(N[C@@H]1CC[C@H](F)C1)[C@H]1C[C@@H]1c1ccc2c(c1)OCCO2
COCC(=O)N(C)CC(=O)NCC1(Nc2nccn3nnnc23)CC1


In [29]:
# DEFINE THE SMILES TOKENS AND MAX_LENGTH
tokens = set()
for s in train_smiles:
    tokens = tokens.union(set(s))
tokens = sorted(list(tokens))
max_length = max(len(s) for s in train_smiles)

In [30]:
# DEFINE AND TRAIN THE MODEL
from moses.vae.model import VAE
from moses.vae.config import get_parser
from moses.vae.trainer import VAETrainer

def train_model(epochs=5):
    config = get_parser().parse_known_args()[0]
    config.log_file = None
    config.model_save = None
    config.lr_n_restarts = 1
    config.lr_n_period = epochs
    print(config)

    trainer = VAETrainer(config)
    model = VAE(trainer.get_vocabulary(train_smiles), config)
    model.cuda(device="cuda")

    model = trainer.fit(model, train_smiles)
    return model

In [31]:
# GENERATE MOLECULES AND TEST IF THEY ARE VALID
import numpy as np

def generate_molecules(model, n_molecules=1000):
    predictions = model.sample(n_molecules, max_length) 
    valid = []

    #using chem from rdkit to ensure generated molecules are valid
    count = 0
    for p in predictions:
      count += 1
      smiles = ''.join(p)
      if count < 10:
        print(smiles)
      if rdkit.Chem.MolFromSmiles(smiles) is not None:
        valid.append(smiles) 

    print(len(valid) / n_molecules)
    return valid

In [32]:
model = train_model(5)
valid = generate_molecules(model, 5000)

print(len(valid), 'valid molecules')
count = 0
for v in valid:
    count += 1
    if count > 20:
        break
    print(v)

Namespace(clip_grad=50, d_cell='gru', d_d_h=512, d_dropout=0, d_n_layers=3, d_z=128, freeze_embeddings=False, kl_start=0, kl_w_end=0.05, kl_w_start=0, log_file=None, lr_end=0.00030000000000000003, lr_n_mult=1, lr_n_period=5, lr_n_restarts=1, lr_start=0.00030000000000000003, model_save=None, n_batch=512, n_jobs=1, n_last=1000, n_workers=1, q_bidir=False, q_cell='gru', q_d_h=256, q_dropout=0.5, q_n_layers=1)


/usr/local/lib/python3.7/dist-packages/moses/vae/model.py:217: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  x[~eos_mask, i] = w[~eos_mask]
/usr/local/lib/python3.7/dist-packages/moses/vae/model.py:219: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  end_pads[i_eos_mask] = i + 1


NC(=O)C(=O)N1CCN(Cc2cnc(Cl)c(C(C)(C)C)n2)C1
Cc1nc[nH]n1CCc1cc([N+](=O)[O-])cnc1[N+](=O)[O-]
Cc1cc(N2CCn2ncnc3[C@@H]2C2CCOCC2)c(C#N)n1
CO[C@@H](Cn1ccnc1)C(=O)NCc1ccc(NC(C)=O)cn1
CCN(CCN1CCCCC1)C(=O)c1ccc2c(n1)CCC2)C1=CNCCC1
CC(C)(N1CCC(C2(CCNCCC3)C2)C(=O)c1ccc(F)cc1
CNC(=O)CCn1cn(C)c(C)c1[N+](=O)[O-]
CCN(C(=O)[C@@H]1C[C@@H]1C1CCSCC1)C(C)C
O=C1CC[C@@H](CNC2(CCNC(=O)CN3CCOCC3)CC2)C1(C)C
0.6036
3018 valid molecules
NC(=O)C(=O)N1CCN(Cc2cnc(Cl)c(C(C)(C)C)n2)C1
CO[C@@H](Cn1ccnc1)C(=O)NCc1ccc(NC(C)=O)cn1
CCN(C(=O)[C@@H]1C[C@@H]1C1CCSCC1)C(C)C
O=C1CC[C@@H](CNC2(CCNC(=O)CN3CCOCC3)CC2)C1(C)C
CCc1ccc(NC(=O)NCCCC2(CO)CCC2)c(C)c1
CCCNC(=O)c1ccn(CCC(F)(F)F)c1NC(=O)[C@H]1[C@@H]2C[C@@H]2C1
CNC(=O)NC(=O)CN(CCNC/C=C/Cl)c1ccccc1
CC[C@H](OC)C(=O)N[C@@H]1CCN(C(=O)[C@@H]2CCN(C)C2=O)C1
CC1(C)O[C@@H]([C@@H](CO)OC(=O)CNC(=O)OCC(C)(C)C(N)=O)CC1
CC(C)c1noc(-c2cccc(Cl)c2)c1N(CC)CC1CC1
CCOC(=O)[C@@H](C1CCCCSC1)c1ccc(Cl)cc1
CCC(=O)N1C[C@](O)([C@@H](C)NC(=O)c2cncnc2)[C@H]1C
c1cnc2nc([C@H](C)C(N)=O)ccc2c1NC(=O)C(F)F
C